In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.cross_validation import train_test_split
import time

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

# Non feature
NON_FEATURE=['activity_id','people_id','date','people_date']

# Categorical data that is only label encoded
CATEGORICAL_DATA = ['people_char_1', 'people_char_2','people_group_1',
                    'people_char_3', 'people_char_4', 'people_char_5',
                    'people_char_6', 'people_char_7', 'people_char_8',
                    'people_char_9', 'activity_category',
                    'char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6',
                    'char_7', 'char_8', 'char_9']

#removed char_10 to check xgb

# Already in a one-hot encoded form
CATEGORICAL_BINARY = ['people_char_10', 'people_char_11', 'people_char_12',
                      'people_char_13', 'people_char_14', 'people_char_15',
                      'people_char_16', 'people_char_17', 'people_char_18',
                      'people_char_19', 'people_char_20', 'people_char_21',
                      'people_char_22', 'people_char_23', 'people_char_24',
                      'people_char_25', 'people_char_26', 'people_char_27',
                      'people_char_28', 'people_char_29', 'people_char_30',
                      'people_char_31', 'people_char_32', 'people_char_33',
                      'people_char_34', 'people_char_35', 'people_char_36',
                      'people_char_37' ]

# Continuous categories
CONT = ['people_days', 'days',
        'people_month',  'month',
        'people_quarter', 'quarter',
        'people_week', 'week',
        'people_dayOfMonth', 'dayOfMonth',
        'people_year', 'year',
        'people_char_38']


# Path to people.csv from ReadHat Kaggle data set with reduced dimensions
FEATURE_FILE ='Data/act_train_features_reduced.csv'
# Path to act_train.csv from RedHat Kaggle data set with reduced dimensions
OUTPUT ='Data/act_train_output.csv'

TEST_FILE = 'Data/act_test_features_reduced.csv'

In [3]:

def category_to_one_hot(dataset, non_feature, continuous_feature):
    # Function to change labels of categories to one-hot encoding using scikit's OneHot Encoding sparse matrix
    # pd.get_dummies(df) does the same, provides sweet header's as well but it kill's memory
    ds = dataset.drop(non_feature, axis=1)
    boolean_column = []
    counter = 0
    for column in ds.columns:
        if column not in continuous_feature:
            boolean_column.append(counter)
        counter += 1
    # boolean_column is not the column name but index
    print("Done filtering columns...")
    grd_enc = OneHotEncoder(categorical_features=boolean_column)
    encoded_arr = grd_enc.fit_transform(ds)
    return encoded_arr

In [4]:
# Read the data set. Note this dataset does not contain the 'outcome' columns
train_data_df = pd.read_csv(FEATURE_FILE,parse_dates=["date"])
train_data_df.sort_values(by=['activity_id'],ascending=True, inplace=True)


# Read the train data output
train_output = pd.read_csv(OUTPUT)
train_output.sort_values(by='activity_id',ascending=True, inplace=True)
v_out=train_output['outcome'].as_matrix()

test_data_df = pd.read_csv(TEST_FILE,parse_dates=["date"])
test_data_df.sort_values(by=['activity_id'],ascending=True, inplace=True)

In [ ]:
#NON_FEATURES=[]
#for column in data_df.columns:
#    if column not in SELECTED_FEATURES:
#        NON_FEATURES.append(column)

In [5]:
############################ UNTOUCHED ######################################
def normalize(df,non_features):
    df=df.drop(non_features,axis=1)
    features=df.columns
    # Normalize categorical values to range betwwen 0-1, time usually : 3secs
    start=time.time()
    scaler = Normalizer()
    df[features] = scaler.fit_transform(df[features])
    end = time.time()
    print (end-start)
    return df

In [6]:
## Try random forest + linear regression
# forest_data=data_df[[ 'people_group_1']].as_matrix()
#X_train, X_test, y_train, y_test = train_test_split(train_data_df,v_out, test_size=0.4, random_state=4)

In [6]:
start = time.time()
## SAMPLE: without dropping char_10
train_arr = category_to_one_hot(train_data_df, NON_FEATURE, CONT)
#train_arr.set_index(['activity_id']).to_csv("act_train_features_onehot.csv")
## SAMPLE: try to run with char_10 first, if it does crash, you add it in NON_FEATURE and then run this code. Okay?
end = time.time()
print(end-start)

Done filtering columns...


AttributeError: set_index not found

In [7]:
start = time.time()
## SAMPLE: without dropping char_10
test_arr = category_to_one_hot(test_data_df, NON_FEATURE, CONT)
## SAMPLE: try to run with char_10 first, if it does crash, you add it in NON_FEATURE and then run this code. Okay?
end = time.time()
print(end-start)

Done filtering columns...
15.6170809269


In [7]:
scal=Normalizer()
arr_train_norm=scal.fit_transform(train_arr)

In [10]:
scal=Normalizer()
arr_test_norm=scal.fit_transform(test_arr)

In [12]:
from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=25,
                    objective='binary:logistic', subsample=0.7,
                    colsample_bytree=0.7, seed=0, silent=1, nthread=4,
                    min_child_weight=0)

watchlist  = [(arr_train_norm,'train')]
num_round = 300
early_stopping_rounds=10

#bst = xgb.train(param, arr_train_norm, num_round, watchlist,early_stopping_rounds=early_stopping_rounds)

xgb.fit(arr_train_norm, v_out,eval_metric='auc')

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.7,
       gamma=0, learning_rate=0.3, max_delta_step=0, max_depth=10,
       min_child_weight=0, missing=None, n_estimators=25, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=1, subsample=0.7)

In [9]:
#Trial. Run instead of above three 
dtrain = xgb.DMatrix(train_arr,label=v_out)
dtest = xgb.DMatrix(test_arr)

param = {'max_depth':10, 'eta':0.02, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['subsample'] = 0.7
param['colsample_bytree']= 0.7
param['min_child_weight'] = 0
param['booster'] = "gblinear"

watchlist  = [(dtrain,'train')]
num_round = 300
early_stopping_rounds=10
bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=early_stopping_rounds)

ypred = bst.predict(dtest)
output = pd.DataFrame({ 'activity_id' : test['activity_id'], 'outcome': ypred })
output.head()
output.to_csv('without_leak.csv', index = False)

[0]	train-auc:0.849711
Will train until train-auc hasn't improved in 10 rounds.
[1]	train-auc:0.856853
[2]	train-auc:0.864064
[3]	train-auc:0.87173
[4]	train-auc:0.880292
[5]	train-auc:0.889228
[6]	train-auc:0.897624
[7]	train-auc:0.905135
[8]	train-auc:0.911837
[9]	train-auc:0.917867
[10]	train-auc:0.923282
[11]	train-auc:0.92811
[12]	train-auc:0.932371
[13]	train-auc:0.936119
[14]	train-auc:0.939419
[15]	train-auc:0.942329
[16]	train-auc:0.944907
[17]	train-auc:0.947196
[18]	train-auc:0.949237
[19]	train-auc:0.95107
[20]	train-auc:0.952724
[21]	train-auc:0.95422
[22]	train-auc:0.955581
[23]	train-auc:0.956822
[24]	train-auc:0.957959
[25]	train-auc:0.959005
[26]	train-auc:0.959967
[27]	train-auc:0.960851
[28]	train-auc:0.961668
[29]	train-auc:0.962422
[30]	train-auc:0.96312
[31]	train-auc:0.963767
[32]	train-auc:0.964365
[33]	train-auc:0.964922
[34]	train-auc:0.965438
[35]	train-auc:0.965916
[36]	train-auc:0.966359
[37]	train-auc:0.966769
[38]	train-auc:0.967152
[39]	train-auc:0.96750

NameError: name 'test' is not defined

In [10]:
output = pd.DataFrame({ 'activity_id' : test_data_df['activity_id'], 'outcome': ypred })
output.head()
output.to_csv('without_leak.csv', index = False)

In [14]:

#y_pred = xgb.predict(arr_test_norm)
#predictions = [round(value) for value in y_pred]

#For predicting probability for final kaggle outcome
#predictions = bst.predict(arr_test_norm)
predictions = xgb.predict_proba(arr_test_norm)

In [15]:
np.savetxt('feature_importance_xgb' + '.txt',
           xgb.feature_importances_, delimiter=',')

In [16]:
test_data_df['outcome']=predictions[:,1]

test_data_df[['outcome','activity_id']].set_index('activity_id').drop('act_0').to_csv("XGBOOST_results.csv")


#from sklearn.metrics import accuracy_score

# evaluate predictions
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))